In [1]:

%pprint

Pretty printing has been turned OFF



## Let's use our labeled data to evaluate a Conditional Random Fields model

In [2]:

%%time
%run ../py/html_analysis.py

crf = CrfUtilities()

Pickling to C:\Users\DaveBabbitt\Documents\Repositories\job-hunting\saves\pickle\CHILDLESS_TAGS_LIST.pickle


AssertionError: You're in deep doodoo: you lost your basic tags dictionary

In [ ]:

files_list = crf.su.get_files_list(crf.su.cursor, verbose=False)
crf.CRF.predict_single(xseq=feature_dict_list)

In [ ]:

[f'crf.CRF.{fn}' for fn in dir(crf.CRF) if 'predict' in fn.lower()]

In [ ]:


import sklearn_crfsuite

CRF = sklearn_crfsuite.CRF(algorithm='lbfgs', c1=0.1, c2=0.1, max_iterations=100,
                           all_possible_transitions=True)
HEADER_PATTERN_DICT = s.load_object('HEADER_PATTERN_DICT')
X_train = []
y_train = []
HEADER_PATTERN_DICT

In [ ]:

crf.CRF.predict_marginals_single(sent_str)

In [ ]:

sent_str = 'You work here.'
crf.CRF.predict_single(sent_str)

In [ ]:

[f'lr.{fn}' for fn in dir(lr) if not fn.startswith('_')]

In [ ]:

import random

print('\nCV')
print(lr.CV)
print('\nLR_DICT')
for k, v in random.sample(lr.LR_DICT.items(), 2):
    print(k, v)
print('\nPREDICT_PERCENT_FIT_DICT')
for k, v in random.sample(lr.PREDICT_PERCENT_FIT_DICT.items(), 2):
    print(k, v)
print('\nTT')
print(lr.TT)


## Test its accuracy

In [ ]:

sql_str = f'''
    SELECT
        np.[navigable_parent],
        pos.[pos_symbol]
    FROM
        [Jobhunting].[dbo].[NavigableParents] np INNER JOIN
        [Jobhunting].[dbo].[PartsOfSpeech] pos ON
            pos.[is_header] = np.[is_header] AND
            pos.[is_task_scope] = np.[is_task_scope] AND
            pos.[is_minimum_qualification] = np.[is_minimum_qualification] AND
            pos.[is_preferred_qualification] = np.[is_preferred_qualification] AND
            pos.[is_legal_notification] = np.[is_legal_notification] AND
            pos.[is_job_title] = np.[is_job_title] AND
            pos.[is_office_location] = np.[is_office_location] AND
            pos.[is_job_duration] = np.[is_job_duration] AND
            pos.[is_supplemental_pay] = np.[is_supplemental_pay] AND
            pos.[is_educational_requirement] = np.[is_educational_requirement] AND
            pos.[is_interview_procedure] = np.[is_interview_procedure] AND
            pos.[is_corporate_scope] = np.[is_corporate_scope] AND
            pos.[is_posting_date] = np.[is_posting_date] AND
            pos.[is_other] = np.[is_other];'''
pos_df = pd.DataFrame(lr.su.get_execution_results(lr.cursor, sql_str, verbose=False))
POS_DICT = pos_df.set_index('navigable_parent')['pos_symbol'].to_dict()

In [ ]:

y_true = []
y_pred = []
for sent_str, pos_symbol in POS_DICT.items():
    y_true.append(pos_symbol)
    y_pred.append(lr.lr_predict_single(sent_str))

In [ ]:

from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred))

In [ ]:

seen = set()
seen_add = seen.add
label_list = [x for x in y_true if not (x in seen or seen_add(x))]

In [ ]:

from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt

cm = confusion_matrix(y_true, y_pred, labels=label_list)
fig, ax = plt.subplots(figsize=(18, 6))
cm_display = ConfusionMatrixDisplay(cm).plot(ax=ax)
xticklabels_list = ax.set_xticklabels(label_list, rotation='vertical')
yticklabels_list = ax.set_yticklabels(label_list)